In [16]:
import pandas as pd
import numpy as np

In [19]:
pp_dep = pd.read_csv("pp_dep.csv", sep=";", dtype={"reg":str})

In [20]:
pp_dep.period_date.unique()

array(['2021-01-31 00:00:00', '2020-12-31 00:00:00',
       '2020-11-30 00:00:00', '2021-02-28 00:00:00',
       '2022-12-31 00:00:00'], dtype=object)

In [21]:
pp_dep.groupby("period_date").count()

,Unnamed: 0,indicateur,period_month_tri,Date,valeur,departement,mesure,short_indic,short_mesure,dep,reg,libelle,region
period_date,,,,,,,,,,,,,
2020-11-30 00:00:00,1995,1995,1995,1995,1995,1995,1995,1995,1995,1995,1995,1995,1995
2020-12-31 00:00:00,3223,3223,3223,3223,3223,3223,3223,3223,3223,3223,3223,3223,3223
2021-01-31 00:00:00,4060,4060,4060,4060,4060,4060,4060,4060,4060,4060,4060,4060,4060
2021-02-28 00:00:00,300,300,300,300,300,300,300,300,300,300,300,300,300
2022-12-31 00:00:00,100,100,100,100,100,100,100,100,100,100,100,100,100


In [22]:
export = pp_dep[["mesure", "short_indic","dep", "valeur"]]
export = pd.pivot_table(export, values="valeur", index=["dep"], columns=["mesure", "short_indic"],aggfunc=np.sum)
export.to_csv("export_20210131.csv", sep=";")
print(export.shape)

(100, 46)
